In [15]:
#US Census API 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

api_key = "9c43c55c9067920bded793b5c4d99ae0aba10281"

c = Census(api_key, year=2017)
api_key

'9c43c55c9067920bded793b5c4d99ae0aba10281'

In [16]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs3.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E","B01002_001E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate"]]

# Visualize
print(len(census_pd))
census_pd.head()

CensusException: <html><head><title>Error report</title><style><!--H1 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:22px;} H2 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:16px;} H3 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:14px;} BODY {font-family:Tahoma,Arial,sans-serif;color:black;background-color:white;} B {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;} P {font-family:Tahoma,Arial,sans-serif;background:white;color:black;font-size:12px;}A {color : black;}A.name {color : black;}HR {color : #525D76;}--></style> </head><body><h1>HTTP Status 404 - /data/2017/acs/acs3</h1><HR size="1" noshade="noshade"><p><b>type</b> Status report</p><p><b>message</b> <u>/data/2017/acs/acs3</u></p><p><b>description</b> <u>The requested resource is not available.</u></p><HR size="1" noshade="noshade"></body></html>

In [ ]:
zips = ['10002','11201','11237','11233','11226',
 '11225','11222','11221','11217','11216','11215','11211','11206','11205','10036','10003',
 '10031','10029','10027','10025','10024','10023','10019','10016','10014','10012','10011',
 '10009','11238']

census_pd2 = census_pd[census_pd["Zipcode"].isin(zips)]
census_pd2

In [ ]:
final30 = pd.read_csv('../Final_Trending_Zipcodes.csv')
final30 = final30.rename(columns={'Zip Codes': 'Zipcode'})

census_pd2['Zipcode']=census_pd2['Zipcode'].astype(int)

zipsmerged = pd.merge(census_pd2, final30, on="Zipcode", how="outer")
zipsmerged